In [ ]:
# Установка Python-библиотек
!pip install -q nltk python-Levenshtein SpeechRecognition pydub > /dev/null

# Импорт библиотек

In [ ]:
import speech_recognition as sr
import requests
import io
from pydub.generators import WhiteNoise
from pydub import AudioSegment
from IPython.display import Audio
import pandas as pd
import warnings
from nltk.translate.bleu_score import sentence_bleu
import Levenshtein
warnings.filterwarnings("ignore")

# Загрузка аудиофайлов

In [ ]:
# Список URL аудиофайлов
audio_urls = [
    "https://storage.yandexcloud.net/datasetsforme/Voice01.mp3",
    "https://storage.yandexcloud.net/datasetsforme/nash-chelovek.mp3",
    "https://storage.yandexcloud.net/datasetsforme/petrosja.mp3",
    "https://storage.yandexcloud.net/datasetsforme/vyfvyf.mp3",
    "https://storage.yandexcloud.net/datasetsforme/Voice02.mp3"
]

In [ ]:
# Функция загрузки и конвертации аудио
def download_and_convert_audio(url):
    response = requests.get(url)
    response.raise_for_status()
    audio_file = io.BytesIO(response.content)
    audio = AudioSegment.from_mp3(audio_file)
    return audio

In [ ]:
# Загрузка всех аудиофайлов
clean_audios = []
for url in audio_urls:

    audio = download_and_convert_audio(url)
    clean_audios.append(audio)
    print(f"Файл успешно загружен: {url.split('/')[-1]}")

# Воспроизведение аудиофайлов

In [ ]:
from IPython.display import Audio

# Вывод чистого аудио
clean_audios = []
for idx, url in enumerate(audio_urls):
    audio = download_and_convert_audio(url)
    clean_audios.append(audio)
    print(f"Чистый файл {idx + 1}:")
    display(Audio(audio.export(format="wav").read(), rate=audio.frame_rate))

# Добавление шума

In [ ]:
# Добавление шума
def add_noise(audio, noise_level=-20):
    noise = WhiteNoise().to_audio_segment(duration=len(audio), volume=noise_level)
    return audio.overlay(noise)

In [ ]:
noisy_audios = [add_noise(audio) for audio in clean_audios]

In [ ]:
# Вывод зашумленного аудио
for idx, noisy_audio in enumerate(noisy_audios):
    print(f"Зашумленный файл {idx + 1}:")
    display(Audio(noisy_audio.export(format="wav").read(), rate=noisy_audio.frame_rate))

# Распознование текста из аудиофайлов

In [ ]:
recognizer = sr.Recognizer()

In [ ]:
# Распознавание текста
def recognize_audio(audio):
    wav_io = io.BytesIO()
    audio.export(wav_io, format="wav")
    wav_io.seek(0)
    with sr.AudioFile(wav_io) as source:
        audio_data = recognizer.record(source)
    return recognizer.recognize_google(audio_data, language="ru-RU")

In [ ]:
recognized_texts_clean = []
recognized_texts_noisy = []

In [ ]:
for idx, (clean_audio, noisy_audio) in enumerate(zip(clean_audios, noisy_audios)):
    print(f"\nРаспознавание файла {idx + 1}:")

    clean_text = recognize_audio(clean_audio)
    print(f"Чистый текст: \n{clean_text}\n")
    recognized_texts_clean.append(clean_text)

    noisy_text = recognize_audio(noisy_audio)
    print(f"Зашумленный текст: \n{noisy_text}")
    recognized_texts_noisy.append(noisy_text)

# Метрики оценки

WER (коэффициент ошибок на уровне слов) измеряет количество ошибок, сделанных при распознавании текста, относительно эталонного текста. Ошибки включают:

* Substitutions (замены): слово в гипотезе отличается от соответствующего слова в эталоне.
* Deletions (удаления): слово отсутствует в гипотезе, но присутствует в эталоне.
* Insertions (вставки): лишнее слово в гипотезе, отсутствующее в эталоне.

WER полезен для оценки точности распознавания речи на уровне слов. Особенно подходит, если важны структурные и лексические совпадения.

In [ ]:
def calculate_wer(reference, hypothesis):
    ref_words = reference.split()
    hyp_words = hypothesis.split()
    substitutions = sum(1 for r, h in zip(ref_words, hyp_words) if r != h)
    deletions = max(0, len(ref_words) - len(hyp_words))
    insertions = max(0, len(hyp_words) - len(ref_words))
    wer = (substitutions + deletions + insertions) / len(ref_words) if ref_words else 1.0
    return wer

BLEU (метрика для оценки машинного перевода) измеряет сходство между гипотезой (распознанным текстом) и эталонным текстом. Она основана на совпадениях n-грамм (последовательностей слов) между текстами

BLEU полезен, если требуется оценить общее сходство текстов, особенно для длинных предложений, где важны совпадения фраз, но плохо работает для коротких текстов, где n-граммы не могут быть репрезентативными

In [ ]:
def calculate_bleu(reference, hypothesis):
    ref = [reference.split()]  # Эталонный текст в виде списка
    hyp = hypothesis.split()   # Распознанный текст в виде списка
    score = sentence_bleu(ref, hyp)
    return score

Расстояние Левенштейна измеряет минимальное количество операций редактирования, необходимых для преобразования одного текста в другой. Эти операции включают:

* Вставка (insertion): добавление символа.
* Удаление (deletion): удаление символа.
* Замена (substitution): замена одного символа на другой.

Levenshtein Distance полезен для оценки точности распознавания текста, особенно если важен учёт мелких изменений (например, пропущенные буквы, лишние пробелы и т.п.).

Метрика не учитывает смысл текста, только буквальные различия. Например, синонимы или перестановка слов могут быть ошибочно посчитаны за значительные изменения.

In [ ]:
def calculate_levenshtein(reference, hypothesis):
    return Levenshtein.distance(reference, hypothesis)

CER (коэффициент ошибок на уровне символов) измеряет долю ошибок, допущенных при преобразовании эталонного текста в гипотезу. Ошибки включают:

* Substitutions (замены): символ в гипотезе отличается от соответствующего символа в эталоне.
* Deletions (удаления): символ присутствует в эталоне, но отсутствует в гипотезе.
* Insertions (вставки): лишний символ присутствует в гипотезе, но отсутствует в эталоне.

CER полезен для оценки качества текста на уровне символов. Это особенно актуально для языков с фиксированным написанием слов (например, для кодов, чисел или технических терминов)

 Но CER не учитывает контекст и может неадекватно оценивать качество, если есть большие перестановки текста или смысловые синонимы.

In [ ]:
def calculate_cer(reference, hypothesis):
    ref_chars = list(reference)
    hyp_chars = list(hypothesis)
    substitutions = sum(1 for r, h in zip(ref_chars, hyp_chars) if r != h)
    deletions = max(0, len(ref_chars) - len(hyp_chars))
    insertions = max(0, len(hyp_chars) - len(ref_chars))
    cer = (substitutions + deletions + insertions) / len(ref_chars) if ref_chars else 1.0
    return cer

In [ ]:
wer = calculate_wer(clean_text, noisy_text)
cer = calculate_cer(clean_text, noisy_text)
bleu = calculate_bleu(clean_text, noisy_text)
levenshtein = calculate_levenshtein(clean_text, noisy_text)

In [ ]:
# Сбор результатов в виде словаря для трансформации
results = {
    "Метрика": ["WER", "CER", "BLEU", "Levenshtein"],
}

# Заполнение данных для каждого файла
for idx, file_name in enumerate(audio_urls):
    clean_text = recognized_texts_clean[idx]
    noisy_text = recognized_texts_noisy[idx]

    # Расчёт метрик
    wer = calculate_wer(clean_text, noisy_text)
    cer = calculate_cer(clean_text, noisy_text)
    bleu = calculate_bleu(clean_text, noisy_text)
    levenshtein = calculate_levenshtein(clean_text, noisy_text)

    # Добавление данных по файлу
    results[file_name.split('/')[-1]] = [round(wer, 2), round(cer, 2), round(bleu, 2), round(levenshtein, 2)]

In [ ]:
# Преобразование результатов в DataFrame
df_results = pd.DataFrame(results)

# Вывод таблицы
print(df_results.to_string(index=False))

# Интерпретация метрик

1. **Voice01.mp3**:
   - **WER (0.88)** и **CER (0.76)**: Высокие значения ошибок — распознавание текста сильно искажено.
   - **BLEU (0.00)**: Совпадение с эталоном отсутствует.
   - **Levenshtein (29.00)**: Множество изменений требуется для совпадения.

2. **nash-chelovek.mp3**:
   - **WER (0.07)** и **CER (0.19)**: Очень низкие ошибки — текст почти идеален.
   - **BLEU (0.87)**: Высокая схожесть с эталоном.
   - **Levenshtein (8.00)**: Минимальные исправления.

3. **petrosja.mp3**:
   - **WER (0.87)** и **CER (0.85)**: Ошибки почти повсеместны.
   - **BLEU (0.59)**: Умеренная схожесть с эталоном.
   - **Levenshtein (178.00)**: Требуется огромное количество правок.

4. **vyfvyf.mp3**:
   - **WER (0.87)** и **CER (0.96)**: Практически полное расхождение.
   - **BLEU (0.55)**: Средний уровень совпадения.
   - **Levenshtein (22.00)**: Много исправлений, но меньше, чем у **petrosja.mp3**.

5. **Voice02.mp3**:
   - **WER (0.08)** и **CER (0.42)**: Низкие ошибки (особенно WER), текст хорошо распознан.
   - **BLEU (0.76)**: Высокая схожесть с эталоном.
   - **Levenshtein (2.00)**: Минимальное количество правок.

**Вывод**: Лучшие результаты у **nash-chelovek.mp3** и **Voice02.mp3**. Худшие — у **petrosja.mp3** и **vyfvyf.mp3**.
